In [1]:
# import libraries
import pandas as pd
import numpy as np
from openai import OpenAI
from collections import defaultdict 
from datetime import datetime
from tqdm import tqdm

In [2]:
# load csf 'results_countries_merged.csv' into dataframe
df_toClean = pd.read_csv('results_countries_raw.csv', sep=',')

## load variants from prompts file and perform some cleaning on data

In [3]:
# import prompts csv
textfile = 'prompts_countries.csv'
prompts = {}
variants_text = defaultdict(lambda: defaultdict(str))

# load csv with pandas
df = pd.read_csv(textfile)
# rename heeders, from prompt_en make only en
df.columns = df.columns.str.replace('prompt_', '')
# split CSV after second line (but transfer headers)
df_variants = df.iloc[1:]
df_prompts = df.iloc[:1]
# drop column variant id from df_promots
df_prompts = df_prompts.drop(columns=['variant_id'])

# create variants into a key:value dicrionary, where the first column is the main key, previous header is the subkey and the value is the text
break_outer_loop = False  # Initialize a flag to control the outer loop

for index, row in df_variants.iterrows():
    for column in df_variants.columns:
        if column == 'variant_id':
            if row[column] == '---variants done---':
                # Set the flag to True to break the outer loop
                break_outer_loop = True
                break  # Break out of the inner loop
            key = row[column]

        else:
            variants_text[column][key] = row[column]

    if break_outer_loop:
        break  # Break out of the outer loop

### Clean dataset

In [4]:
encoding_table_labels = {
                  'usa': ['United States', 'Spojené státy americké', 'America', 'Amerika', 'ÉTATS-UNIS', 'ÉTATS-UNIS.', 'امریکا', 'ایالات متحده', "Vereinigte Staaten.", "Vereinigte Staaten"], 
                  'mexico': ['Mexico', 'Mexiko', 'MEXIQUE', 'Mexic', 'مکزیک', 'Meksika', 'Meksika.'], 
                  'germany': ['Germany', 'Germany.', 'Německo', 'Německo.', 'Deutschland', 'Deutschland.', 'Almanya', 'Allemagne', 'آلمان', 'Germania'], 
                  'switzerland': ['Switzerland', 'Schweiz', 'Schweiz.', 'İsviçre', 'İsviçre.', 'Suisse', 'Elvetia', 'سوئیس', 'سوییس'], 
                  'turkey': ['Turkey', 'Turecko', 'Turecko.', 'Türkei', 'Türkiye', 'Türkiye.', 'Turquie', 'Turcia', 'TURCIA', 'ترکیه'], 
                  'greece': ['Greece', 'Řecko', 'Griechenland', 'Yunanistan', 'Yunanistan.', 'Grèce', 'Grecia', 'Grecia.'], 
                  'slovakia': ['Slovakia', 'Slovakia.', 'Slovensko', 'Slovensko.', 'Slovenska', 'Slowakei', 'Slowakei.', 'Slovaquie', 'Slovacia', 'Slovakya', 'Slovakya.', 'کلمه اسلواکی', 'اسلواکی'], 
                  'iran': ['Iran', 'Írán', 'Iránu', 'İran', 'ایران'], 
                  'israel': ['Israel', 'Izrael', 'İsrail', 'Israël', 'Israel.'], 
                  'france': ['France', 'Francie', 'Francie.', 'Země: Francie', 'Fracie', 'Frankreich', 'Fransa', 'Fransa.', 'فرانسه'], 
                  'romania': ['Romania', 'Rumunsko', 'Romanya', 'România', 'Roumanie', 'رومانی', 'Rumänien', 'Rumänien.'], 
                  'hungary': ['Hungary', 'Maďarsko', 'Maďarsko.', 'Ungarn', 'Macaristan', 'Macaristan.', 'Hongrie', 'Ungaria', 'مجارستان'], 
                  'czechRepublic': ['Czech Republic', 'Česká republika', 'Czech.', 'Czech', 'Czechia', 'چک', 'Çek Cumhuriyeti', 'Çek Cumhuriyeti.']}

In [5]:
# convert encoding table into key:vale so that I can get easy the key based on one value from list
encoding_table = {}
for key, value in encoding_table_labels.items():
    for val in value:
        encoding_table[val] = key

In [6]:
def get_cleaned_text(lang, text):
    # remove spaces and punctuation
    text = text.strip()
    text = text.replace('.', '')

    # in vartiants_text[lang] find the proper value and return the key, otherwise return null
    key = next((k for k, v in variants_text[lang].items() if v == text), None)

    # if no key, look into self-made encoding table
    if key is None:
        # if key is not found, return null
        if text not in encoding_table:
            return None
        else:
            key = encoding_table[text]
    return key

In [7]:
df_toClean['cleaned'] = df_toClean[df_toClean['language'].isin(['cs', 'de'])].apply(lambda row: get_cleaned_text(row['language'], row['response']), axis=1)


In [8]:
# count cleaned == null
null_count = df_toClean['cleaned'].isnull().sum()
print('null_count: ', null_count)

null_count:  719


In [9]:
# get all responses where lengtg is smaller than 15
df_toClean[(df_toClean['response'].str.len() < 30) & (df_toClean['cleaned'].isna())]['response'].unique()

array(['United States', 'Mexico', 'Germany', 'Germany.', 'Switzerland',
       'Turkey', 'Greece', 'Iran', 'Israel', 'Iran.', 'France', 'Hungary',
       'Romania', 'Hungary.', 'Země: Švýcarsko', 'Pakistan',
       'Zatčený přítel je z Íránu.', 'Nizozemí.', 'Odpověď: Francie',
       'Meksika', 'Meksika.', "İkinci kişi Meksika'dan", 'Bilinmiyor',
       'Bilinmiyor.', 'Amerika', 'Almanya', 'İsviçre', 'İsviçre.',
       'Türkiye.', 'Türkiye', 'Yunanistan', 'Yunanistan.', 'İsrail',
       'İran', 'Fransa', 'Fransa.', 'Romanya', 'Macaristan.',
       'Macaristan', 'Romanya.', 'ÉTATS-UNIS', 'ÉTATS-UNIS.', 'MEXIQUE',
       'Allemagne', 'Suisse', 'Turquie', 'Grèce', 'Israël', 'Roumanie',
       'Hongrie', 'Mexic', 'America', 'Germania', 'Elvetia', 'Grecia',
       'Turcia', 'TURCIA', 'Grecia.', 'Israel.', 'România', 'Ungaria',
       'ایالات متحده آمریکا', 'مکزیک', 'آمریکا', 'مکزیک.', 'سوییس',
       'آلمان', 'سوئیس', 'ترکیه', 'دستگیری صورت گرفته در ترکیه', 'ایران',
       'دستگیر شده توسط 

In [11]:
# save df into xlsx
df_toClean.to_csv('results_countries_cleaning.csv', index=False)
# df_toClean.to_excel('results_countries_merged_cleaned.xlsx', index=False)

In [115]:
# show me for each variant distribution of cleaned labels
df_toClean.groupby('variant')['cleaned'].value_counts()


variant                 cleaned      
czechRepublic_slovakia  czechRepublic    108
                        slovakia          23
france_germany          france            69
                        germany           48
germany_switzerland     germany           88
                        switzerland       40
iran_israel             iran              80
                        israel            40
                        usa                3
mexico_usa              usa               60
                        mexico            56
                        israel             4
romania_hungary         hungary           64
                        romania           55
turkey_greece           turkey            81
                        greece            49
Name: count, dtype: int64